In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import random

from IPython.display import display, HTML

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

In [2]:
class Person:
    def __init__(self):
        self.recover_prob = 0.2
        self.die_prob = 0.05
        self.init_sick_prob = 0.1
        self.infect_others_prob = 0.05
        self.vaccinated = False
        self.recovered = False
        self.dead = False
        self.sick = False
        
        self.average_meetups = 10
        self.days_sick = 0
        self.init_sick_or_not()
        
    def init_sick_or_not(self):
        """When created each person starts as either sick or healthy"""
        prob = random.random()
        if prob <= self.init_sick_prob:
            # This should yield that about 10% of the people created are sick, the rest healthy
            self.sick = True
        else:
            self.sick = False
    
    def day_passes(self, population, init_scenario):
        """This method describes what happends to each person each day"""
        if self.sick == True:
            self.days_sick += 1
            if init_scenario == False:
                self.infect_others(population)
        
        # Will person recover?
        prob = random.random()
        if prob <= self.recover_prob and self.sick == True:
            # Person has rehabilitated and is now healthy!
            self.sick = False
            self.recovered = True 
        
        # If person is still sick they might die
        prob = random.random()
        if prob <= self.die_prob and self.sick == True:
            self.dead = True
            self.sick = False
            
    def infect_others(self, population):
        person_encounters = random.sample(range(population.size), self.average_meetups)
        for person_id in person_encounters:
            prob = random.random()
            person = population[person_id]
            if prob <= self.infect_others_prob and person.dead == False and person.recovered == False and person.vaccinated == False:
                population[person_id].sick = True  

In [5]:
class Village:
    def __init__(self, init_population_size):
        self.population = np.empty(init_population_size, Person)
        self.init_vaccination = 0.2 * init_population_size
        self.daily_vaccination_threshold = 0.04 * init_population_size
        self.vaccination_started = False
        self.generate_inhabitants()
        
        
    def advance_days(self, init_scenario = False):
        """Counts the status of the citizens in the community"""
        people_sick = 0
        people_recovered = 0
        people_dead = 0
        people_vaccinated = 0     
        people_immune = 0
        
        for person in self.population:
            if person.sick == True:
                people_sick += 1
            if person.dead == True:
                people_dead += 1
                
            if person.recovered == True and person.vaccinated == True:
                people_recovered += 1
                people_vaccinated += 1
                people_immune += 1
            elif person.vaccinated == True:
                people_vaccinated += 1
                people_immune += 1
            elif person.recovered == True:
                people_recovered += 1 
                people_immune += 1
            
            person.day_passes(self.population, init_scenario)
        
        people_susceptible = self.population.size - (people_immune + people_dead + people_sick)
        
        if people_sick >= self.init_vaccination and self.vaccination_started == False:
            print(f"Vaccination has started! At the end of the day {people_sick} are sick and the community is on the alert")
            self.vaccination_started = True
            
        if self.vaccination_started == True:
            self.vaccinate_population()
            
        return people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible
    
    def generate_inhabitants(self):
        """Generates the population"""
        for i in range(self.population.size):
            self.population[i] = Person()
            
    def vaccinate_population(self):
        people_vaccinated_today = 0
        
        for person in self.population:
            if people_vaccinated_today == self.daily_vaccination_threshold:
                break
            elif person.sick == False and person.dead == False and person.vaccinated == False:
                person.vaccinated = True
                people_vaccinated_today += 1
                
    # Assignment 5a - 3.2 Creating the data frame and saving as csv file
    def gen_df_and_save(self, day_data_array):
        columns = ['Sick', 'Recoverd', 'Deceased', 'Vaccinated', 'Immune', 'Susceptible']
        df = pd.DataFrame(day_data_array, columns = columns)
        display(df)
        df.to_csv('my_ass5_dataset.csv', index = False)
        
    
    def start_simulation(self):
        """This function controls the simulation and what happends in a day"""
        
        current_day = 0
        day_data_list = []
    
        people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible = self.advance_days(init_scenario = True)
        
        # Assignment 5a - Generating the dataset
        day_data = [people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible]
        day_data_list.append(day_data)

        
        while people_sick != 0:
            print(f"By day {current_day} {people_sick} people are sick, {people_recovered} has recovered and {people_dead} are dead.")
            current_day += 1
            
            people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible = self.advance_days()
            # Assignment 5a - Generating the dataset
            day_data = [people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible]
            day_data_list.append(day_data)
        
        #Assignment 5a - Generating the dataset
        day_data_array = np.array(day_data_list)
        
        # The village is free of the virus, simulation ended
        days_sick = [person.days_sick for person in self.population]
        
        # Assignment 5a - 3.2 Creating the data frame and saving as csv file
        self.gen_df_and_save(day_data_array)
        
        
        print("\n-------END OF SIMULATION-------")
        print(f"By day {current_day} {people_sick} people are sick, {people_recovered} has recovered and {people_dead} are dead.")
        print(f"In total {people_vaccinated} people recieved vaccination and {people_susceptible} remain susceptible to the virus.")
        print("--------------")
        print("The village has recovered and the virus is eliminated!")
        print("The longest time an individual was sick is: ", max(days_sick), "days")
        
        """Assignment 5a"""
        print(day_data_array)

In [6]:
def main():
    pop_size = 1000
    village = Village(pop_size)
    village.start_simulation()

main()

By day 0 100 people are sick, 0 has recovered and 0 are dead.
By day 1 106 people are sick, 15 has recovered and 3 are dead.
By day 2 131 people are sick, 49 has recovered and 8 are dead.
By day 3 166 people are sick, 71 has recovered and 14 are dead.
By day 4 195 people are sick, 97 has recovered and 23 are dead.
Vaccination has started! At the end of the day 209 are sick and the community is on the alert
By day 5 209 people are sick, 146 has recovered and 30 are dead.
By day 6 217 people are sick, 185 has recovered and 37 are dead.
By day 7 221 people are sick, 217 has recovered and 44 are dead.
By day 8 201 people are sick, 269 has recovered and 58 are dead.
By day 9 188 people are sick, 312 has recovered and 67 are dead.
By day 10 179 people are sick, 347 has recovered and 76 are dead.
By day 11 141 people are sick, 395 has recovered and 84 are dead.
By day 12 113 people are sick, 422 has recovered and 91 are dead.
By day 13 94 people are sick, 446 has recovered and 98 are dead.
By

,Sick,Recoverd,Deceased,Vaccinated,Immune,Susceptible
0,100,0,0,0,0,900
1,106,15,3,0,15,876
2,131,49,8,0,49,812
3,166,71,14,0,71,749
4,195,97,23,0,97,685
5,209,146,30,0,146,615
6,217,185,37,40,215,531
7,221,217,44,80,278,457
8,201,269,58,120,352,389
9,188,312,67,160,414,331



-------END OF SIMULATION-------
By day 30 0 people are sick, 543 has recovered and 114 are dead.
In total 886 people recieved vaccination and 0 remain susceptible to the virus.
--------------
The village has recovered and the virus is eliminated!
The longest time an individual was sick is:  23 days
[[100   0   0   0   0 900]
 [106  15   3   0  15 876]
 [131  49   8   0  49 812]
 [166  71  14   0  71 749]
 [195  97  23   0  97 685]
 [209 146  30   0 146 615]
 [217 185  37  40 215 531]
 [221 217  44  80 278 457]
 [201 269  58 120 352 389]
 [188 312  67 160 414 331]
 [179 347  76 200 469 276]
 [141 395  84 240 534 241]
 [113 422  91 280 574 222]
 [ 94 446  98 320 616 192]
 [ 74 468 100 360 655 171]
 [ 57 481 106 400 679 158]
 [ 53 489 107 440 701 139]
 [ 44 499 109 480 723 124]
 [ 39 506 109 520 740 112]
 [ 28 517 111 560 765  96]
 [ 22 523 111 600 782  85]
 [ 18 527 111 640 800  71]
 [ 16 529 111 680 813  60]
 [ 11 533 112 720 832  45]
 [  8 537 112 760 844  36]
 [  7 537 113 800 856  2